In this file, we perfrom analyzings under RQs

### RQ1 Merge method and its proportion

- What are the Methods of merge in the selected projects?
- What are the 

github上存在的merge途径，及其在PR的占比。

notes：

1. 是否合并到主线
2. base不是default branch，是否最终合并到主线
2. 是否是机器人。感觉这个没有影响。

results：

不同合并的方法：
- Approval event
- github  merge
- commit_mention_merge: 我们在2022年9月clone项目，大约在PR获取的1年之后，使用其default commit。https://stackoverflow.com/a/6742691
- commit_include merge
- comment merge

what if dest branch is not master:
- 可以先计算下分支占比。


1. 还可以计算下不同merge方法的合并联系，是否某种merge方式和其余merge方式强相关。
2. 另外，判断下，合并到主分支的结果、以及合并到从分支的结果。我们猜测，部分PR首先合并到从分支，一段时间后会再合并到主分支中去。这就可能导致了，default branch commit mention且github merge到从分支。



In [1]:
from dbs.sqlite_base import conn, cursor
from config.configs import repos
import json
import pandas as pd

In [2]:
from dbs.tables.projects import get_project_by_full_name
from dbs.tables.prs import list_pr


datas = []
for owner, repo, lang in repos:
    repo_id = get_project_by_full_name(f"{owner}/{repo}")
    prs = list_pr(repo_id)

    sql = f'''--sql
        select p.id, p.url, p.base_label, p.head_label, pmd.merge, pmd.github_merge, pmd.comment_merge, pmd.commit_mention_merge, pmd.commit_include_merge, pmd.approved_merge
        from prs p
        left join pr_merge_decision pmd
        on p.id = pmd.pr_id
        where p.base_repo_id = {repo_id}
    ;'''

    sql = f'''--sql
        select count(p.id) as total, sum(pmd.merge) as merge, sum(pmd.github_merge) as github_merge, 
                sum(pmd.comment_merge) as comment_merge, sum(pmd.commit_mention_merge) as commit_mention_merge, 
                sum(pmd.commit_include_merge) as commit_include_merge, sum(pmd.approved_merge) as approved_merge
        from prs p
        left join pr_merge_decision pmd
        on p.id = pmd.pr_id
        where p.base_repo_id = {repo_id}
    ;'''
    cursor.execute(sql)
    res = cursor.fetchone()

    cur = [owner, repo, res['total']]
    cur.append(res['merge'] / res['total'])
    cur.append(res['github_merge'] / res['total'])
    cur.append(res['comment_merge'] / res['total'])
    cur.append(res['commit_mention_merge'] / res['total'])
    cur.append(res['commit_include_merge'] / res['total'])
    cur.append(res['approved_merge'] / res['total'])
    datas.append(cur)
    
pd.DataFrame(datas, columns=["owner", "repo", "count", "merge", "github_merge", "comment_merge", "commit_mention_merge", "commit_include_merge", "approved_merge"])

,owner,repo,count,merge,github_merge,comment_merge,commit_mention_merge,commit_include_merge,approved_merge
0,atom,atom,5120,0.777734,0.760352,0.000586,0.014648,0.563867,0.083203
1,flutter,flutter,30150,0.738176,0.722653,0.000232,0.086468,0.042056,0.438375
2,nodejs,node,25113,0.848246,0.128658,0.569984,0.731414,0.101979,0.714849
3,kubernetes,kubernetes,64954,0.771700,0.757967,0.000015,0.007575,0.615836,0.140546
4,angular,angular,19416,0.769056,0.154460,0.020756,0.579677,0.037855,0.576844
5,mrdoob,three.js,11664,0.790038,0.759945,0.000257,0.025291,0.628515,0.159808
6,puppeteer,puppeteer,2421,0.835192,0.821974,0.000000,0.202396,0.000413,0.739364
7,vercel,next.js,9006,0.820009,0.801466,0.000111,0.174550,0.002776,0.656895
8,tensorflow,models,3368,0.679038,0.651128,0.000000,0.060570,0.253563,0.541271
9,mui-org,material-ui,14207,0.826494,0.819174,0.000000,0.134300,0.111987,0.354755


# 采样commit mention的文本
sql = '''--sql
    select p.id, p.url, p.base_label, p.head_label, pmd.merge, pmd.github_merge, pmd.comment_merge, pmd.commit_mention_merge, pmd.commit_include_merge, pmd.approved_merge, pmd.commit_mention_msg
    from prs p
    left join pr_merge_decision pmd
    on p.id = pmd.pr_id
    where pmd.commit_mention_merge = 1 and pmd.github_merge = 0 and p.base_repo_id = 25
;'''

cursor.execute(sql)
res = cursor.fetchall()
for pr in res[:100]:
    print(pr['url'])
    print(pr['commit_mention_msg'])
    print("#" * 10)



In [ ]:
# 采样comment mention的文本
sql = '''--sql
    select p.id, p.url, p.base_label, p.head_label, pmd.merge, pmd.github_merge, pmd.comment_merge, pmd.commit_mention_merge, pmd.commit_include_merge, pmd.approved_merge, pmd.comment_msg
    from prs p
    left join pr_merge_decision pmd
    on p.id = pmd.pr_id
    where pmd.comment_merge = 1 and pmd.github_merge = 0 and pmd.commit_mention_merge = 0 and pmd.commit_include_merge = 0 and pmd.approved_merge = 0
;'''

cursor.execute(sql)
res = cursor.fetchall()
for pr in res[:100]:
    print(pr['url'])
    print(pr['comment_msg'])
    print("#" * 10)
'''
# 找的几个例子
atom/atom/4898:Merged this via a787eb9674b09d3f9d5dedc3b107d6892a0357dc with conflicts resolved. Thanks a ton @!
flutter/flutter:Looks like this was included as part of 3a674421299e9a90700e7e23b6112f04c7687c61.
flutter/flutter/1779:Commits squashed and merged in 6bb7c0ae9c87f2ae1d100b5e4818a17e297d2b8b
nodejs/node/39752:Landed in e1e669b1095f
'''


In [ ]:
# 检测差别显著性关系的方法
# Mann–Whitney test
# 好像主要用的是这个python包 statsmodels.stats.multitest.

# 基于Cliff’s delta进行判断



# take off

在这些热门项目中，采用的主要合并方式为github merge，我们观察到merge和github merge得到的比例比较相近。

不过，在node、angular、pytorch、sprint-project等项目中，虽然其合并的PR比例过半，但是github merge信息显示其合并比例较低。说明其并不完全参考github提供的合并逻辑进行合作开发。

~~我们发现每个项目的commit mention处于较高的水平。因此，我们对每个项目进行手工采样调查。。~~ 因为github merge 同样会生成commit mention，这不应该算进来。

"source code PR or config edit PR"


### RQ2 Links between early features and merge decision result




### RQ3 Deep Understanding in PR type and links with merge decision result

PR description would edit by author or others, 
which features would related to PR edit count and can we use first PR description to predict merge decision result

PR type 相关。

### RQ4 about code diff, its 编辑距离和merge decision间的联系